In [ ]:
import numpy as np
import pygame
import random
import time

# Размеры клетки и дополнительная высота для вывода текста (расширено для нескольких строк)
CELL_SIZE = 50
TEXT_AREA_HEIGHT = 150  # увеличенная высота для вывода статистики

# Инициализация pygame
pygame.init()

# Загрузка изображений
TREE_IMAGE = pygame.image.load('tree.png')  # Замените на путь к вашему изображению дерева
FIRE_IMAGE = pygame.image.load('fire.png')  # Замените на путь к вашему изображению огня

# Масштабируем изображения до нужного размера
TREE_IMAGE = pygame.transform.scale(TREE_IMAGE, (CELL_SIZE, CELL_SIZE))
FIRE_IMAGE = pygame.transform.scale(FIRE_IMAGE, (CELL_SIZE, CELL_SIZE))

# Цвет для пустых клеток
EMPTY_COLOR = (255, 255, 255)  # Белый — пустая клетка

# Функция для инициализации поля
def create_forest(n):
    return np.full((n, n), None)  # массив n x n, заполненный значением None

# Функция для расчёта текущей плотности деревьев
def density_trees(forest):
    n = forest.shape[0]
    total_cells = n * n
    tree_count = sum(1 for i in range(n) for j in range(n) if forest[i, j] == 'Tree')
    return tree_count / total_cells

# Функция для получения соседей клетки (четырёх смежных)
def get_neighbors(i, j, n):
    neighbors = []
    if i > 0:         neighbors.append((i - 1, j))  # Верх
    if i < n - 1:     neighbors.append((i + 1, j))  # Низ
    if j > 0:         neighbors.append((i, j - 1))  # Лево
    if j < n - 1:     neighbors.append((i, j + 1))  # Право
    return neighbors

# Функция для подсчёта числа кластеров деревьев (связных компонент по 4-смежности)
def cluster_count(forest):
    n = forest.shape[0]
    visited = np.full((n, n), False)
    clusters = 0
    
    def dfs(i, j):
        stack = [(i, j)]
        while stack:
            ci, cj = stack.pop()
            if visited[ci, cj]:
                continue
            visited[ci, cj] = True
            for ni, nj in get_neighbors(ci, cj, n):
                if not visited[ni, nj] and forest[ni, nj] == 'Tree':
                    stack.append((ni, nj))
    
    for i in range(n):
        for j in range(n):
            if forest[i, j] == 'Tree' and not visited[i, j]:
                clusters += 1
                dfs(i, j)
    return clusters

# Функция для обновления состояния клеток. Возвращает текущую плотность.
def update_forest(forest, epsilon, theta):
    n = forest.shape[0]
    
    # Списки для обработки распространения огня
    fire_spread = []
    to_burn = []

    # Распространение огня от текущих горящих деревьев
    for i in range(n):
        for j in range(n):
            if forest[i, j] == 'Fire':
                for ni, nj in get_neighbors(i, j, n):
                    if forest[ni, nj] == 'Tree':
                        fire_spread.append((ni, nj))
                to_burn.append((i, j))
    
    for ni, nj in fire_spread:
        if forest[ni, nj] == 'Tree':
            forest[ni, nj] = 'Fire'
    
    # Горящие деревья становятся пустыми
    for i, j in to_burn:
        forest[i, j] = None

    # Удары молнии: отдельное возгорание дерева с вероятностью theta
    for i in range(n):
        for j in range(n):
            if forest[i, j] == 'Tree' and random.random() < theta:
                forest[i, j] = 'Fire'
    
    # Появление деревьев в пустых клетках с вероятностью epsilon
    for i in range(n):
        for j in range(n):
            if forest[i, j] is None:
                if random.random() < epsilon:
                    forest[i, j] = 'Tree'

    # Вычисление текущей плотности деревьев
    density = density_trees(forest)
    print("Текущая плотность деревьев:", density)
    return density

# Функция для рисования поля
def draw_forest(screen, forest, n):
    for i in range(n):
        for j in range(n):
            if forest[i, j] == 'Tree':
                screen.blit(TREE_IMAGE, (j * CELL_SIZE, i * CELL_SIZE))
            elif forest[i, j] == 'Fire':
                screen.blit(FIRE_IMAGE, (j * CELL_SIZE, i * CELL_SIZE))
    # Рисуем разделительную линию между полем и областью для текста
    pygame.draw.line(screen, (0, 0, 0), (0, n * CELL_SIZE), (n * CELL_SIZE, n * CELL_SIZE), 2)

# Основная функция игры
def main():
    # Получаем параметры симуляции от пользователя
    n = int(input("Введите размер поля n: "))
    epsilon = float(input("Введите вероятность появления дерева ε (от 0 до 1): "))
    theta = float(input("Введите вероятность удара молнии θ (от 0 до 1): "))

    # Размер окна: высота расширена для вывода дополнительного текста
    width = n * CELL_SIZE
    height = n * CELL_SIZE + TEXT_AREA_HEIGHT
    screen = pygame.display.set_mode((width, height))
    pygame.display.set_caption("Лесной Пиздец")

    # Инициализация поля
    forest = create_forest(n)
    
    # Инициализация шрифта для вывода текста
    font = pygame.font.SysFont("arial", 24)

    # Переменные для накопления статистики
    total_density = 0.0
    total_clusters = 0
    step_count = 0

    # Главный цикл симуляции
    running = True
    clock = pygame.time.Clock()
    while running:
        screen.fill(EMPTY_COLOR)
        
        # Обновляем состояние леса и получаем текущую плотность
        current_density = update_forest(forest, epsilon, theta)
        # Вычисляем число кластеров
        current_clusters = cluster_count(forest)
        print("Число кластеров:", current_clusters)
        
        # Обновляем статистику за все время
        step_count += 1
        total_density += current_density
        total_clusters += current_clusters
        
        avg_density = total_density / step_count
        avg_clusters = total_clusters / step_count
        
        # Рисуем поле
        draw_forest(screen, forest, n)

        # Формируем строки для вывода статистики
        text_current_density = font.render(f"Текущая плотность: {current_density:.2f}", True, (0, 0, 0))
        text_avg_density = font.render(f"Средняя плотность (за все время): {avg_density:.2f}", True, (0, 0, 0))
        text_current_clusters = font.render(f"Число кластеров: {current_clusters}", True, (0, 0, 0))
        text_avg_clusters = font.render(f"Среднее число кластеров (за все время): {avg_clusters:.2f}", True, (0, 0, 0))
        
        # Выводим строки в области для текста
        text_y = n * CELL_SIZE + 10
        screen.blit(text_current_density, (10, text_y))
        screen.blit(text_avg_density, (10, text_y + 30))
        screen.blit(text_current_clusters, (10, text_y + 60))
        screen.blit(text_avg_clusters, (10, text_y + 90))
        
        # Обновляем экран
        pygame.display.flip()

        # Обработка событий (выход по нажатию на крестик)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Пауза для наглядности симуляции
        time.sleep(0.1)
        clock.tick(10)

    pygame.quit()

if __name__ == "__main__":
    main()


Введите размер поля n:  10
Введите вероятность появления дерева ε (от 0 до 1):  0.4
Введите вероятность удара молнии θ (от 0 до 1):  0.2


Текущая плотность деревьев: 0.45
Число кластеров: 10
Текущая плотность деревьев: 0.56
Число кластеров: 8
Текущая плотность деревьев: 0.4
Число кластеров: 20
Текущая плотность деревьев: 0.31
Число кластеров: 18
Текущая плотность деревьев: 0.32
Число кластеров: 20
Текущая плотность деревьев: 0.27
Число кластеров: 18
Текущая плотность деревьев: 0.33
Число кластеров: 16
Текущая плотность деревьев: 0.36
Число кластеров: 13
Текущая плотность деревьев: 0.38
Число кластеров: 14
Текущая плотность деревьев: 0.39
Число кластеров: 18
Текущая плотность деревьев: 0.37
Число кластеров: 16
Текущая плотность деревьев: 0.38
Число кластеров: 18
Текущая плотность деревьев: 0.38
Число кластеров: 17
Текущая плотность деревьев: 0.3
Число кластеров: 16
Текущая плотность деревьев: 0.32
Число кластеров: 18
Текущая плотность деревьев: 0.43
Число кластеров: 9
Текущая плотность деревьев: 0.39
Число кластеров: 9
Текущая плотность деревьев: 0.35
Число кластеров: 10
Текущая плотность деревьев: 0.33
Число кластеров: 1